In [664]:
import plotly.express as px
import pandas as pd
import math

In [665]:
m_b = 19487171
m_s = 10000000
min_delta_b = 1
min_delta_s = 9503
max_delta_b = 6858
max_delta_s = 9835
stepping = "buying"
slopes = [[
        (4877, 9504),
        (4878, 9506),
        (4878, 9506),
        (4879, 9508),
        (4879, 9508),
        (4880, 9510),
        (4880, 9510),
        (4881, 9512),
        (4881, 9512),
        (4882, 9514),
        (4882, 9514),
        (4883, 9516),
        (4883, 9516),
        (4884, 9518),
        (4884, 9518),
        (4885, 9520),
        (4885, 9520),
        (4886, 9522),
        (4886, 9522),
        (4887, 9524),
        (4887, 9524),
        (4888, 9526),
        (4888, 9526),
        (4889, 9528),
        (4889, 9528),
        (4890, 9530),
        (4890, 9530),
        (4891, 9532),
        (4891, 9532),
        (4892, 9534),
        (4892, 9534),
        (4893, 9536),
        (4893, 9536),
        (4894, 9538),
        (4894, 9538),
        (4895, 9539),
        (4913, 9575),
        (4912, 9573),
        (4930, 9608),
        (4948, 9643),
        (4966, 9678),
        (4984, 9713),
        (5002, 9748),
        (5020, 9783),
        (5038, 9818),
        (5056, 9853),
]]

In [666]:
slopes_ = []
for slope in slopes:
    slope_ = []
    slope__ = []
    prevloss = None
    for (delta_b, delta_s) in slope:
        # if stepping == "buying":
        #     loss = (-m_b * buying % m_s) / m_s
        # else:
        #     loss = (m_s * selling % m_b) / m_b
        loss = delta_s * m_s / m_b - math.floor(delta_s * m_s / m_b)
        slope__.append((delta_b, delta_s, loss))
        if prevloss is not None and prevloss < loss:
            prevloss = loss
            loss = -loss
        else:
            prevloss = loss

        slope_.append((delta_b, delta_s, loss))
    slopes_.append(slope_)
    slopes_.append(slope__)
slopes = slopes_

In [667]:
perfect = []
best_imperfect = []
other_imperfect = []
delta_b_too_low = []
suboptimal_delta_b = []
prev_delta_b = None
best_imperfect_loss = None
for delta_s in range(min_delta_s, max_delta_s + 1):
    delta_b = math.floor(delta_s * m_s / m_b)
    # if (delta_b == 0):
    #     print("delta_b == 0 for", delta_s)
    #     continue
    loss = delta_s * m_s / m_b - math.floor(delta_s * m_s / m_b)
    if (delta_b > max_delta_b):
        delta_b = max_delta_b
    if (delta_b < min_delta_b):
        delta_b_too_low.append((delta_b, delta_s, loss))
        continue
    if (delta_b == prev_delta_b):
        suboptimal_delta_b.append((delta_b, delta_s, loss))
        continue
    prev_delta_b = delta_b
    if delta_b * m_b == delta_s * m_s:
      perfect.append((delta_b, delta_s, loss))
    elif best_imperfect_loss is None or loss <= best_imperfect_loss:
      best_imperfect_loss = loss
      best_imperfect.append((delta_b, delta_s, loss))
    else:
      other_imperfect.append((delta_b, delta_s, loss))
    if (delta_b == max_delta_b):
        break
    
delta_b_too_low = []

In [668]:
# Convert your data lists to Pandas DataFrames for easier handling
df1 = pd.DataFrame(perfect, columns=['delta_b', 'delta_s', 'loss'])
df2 = pd.DataFrame(best_imperfect, columns=['delta_b', 'delta_s', 'loss'])
df3 = pd.DataFrame(other_imperfect, columns=['delta_b', 'delta_s', 'loss'])
df4 = pd.DataFrame(delta_b_too_low, columns=['delta_b', 'delta_s', 'loss'])
df5 = pd.DataFrame(suboptimal_delta_b, columns=['delta_b', 'delta_s', 'loss'])
# df6 = pd.DataFrame(first_slope, columns=['delta_b', 'delta_s', 'loss'])
# df7 = pd.DataFrame(second_slope, columns=['delta_b', 'delta_s', 'loss'])
slopes_df = [pd.DataFrame(slope, columns=['delta_b', 'delta_s', 'loss']) for slope in slopes]
for i in range(len(slopes_df)):
    slopes_df[i]['Series'] = 'slope' + str(i + 1)

# Combine the dataframes with an identifier column
df1['Series'] = 'Without'
df2['Series'] = 'best'
df3['Series'] = 'other'
df4['Series'] = 'too low'
df5['Series'] = 'subopt'
# df6['Series'] = '1st'
# df7['Series'] = '2nd'
# pd.concat([df5, df1, df3] + slopes_df + [df2])
combined_df = pd.concat([df4, df5, df1, df3, df2]) 

# # Add a new column for point size based on the series
# # Assuming 'Series' is a column that identifies each series
# series_size_mapping = {series: i+1 for i, series in enumerate(combined_df['Series'].unique())}
# combined_df['point_size'] = combined_df['Series'].map(series_size_mapping)

# # Sort the DataFrame in reverse order of point size
# combined_df = combined_df.sort_values(by='point_size', ascending=False)

In [669]:

# Create an interactive scatter plot
fig = px.scatter(combined_df, x='delta_b', y='loss', color='Series', #size='point_size',
                 hover_data=['delta_b', 'delta_s', 'loss'])

# Add lines for specified series
# for series in slopes_df:
#     fig.add_trace(px.line(series, x='delta_s', y='loss', color='Series').data[0])

# Concatenate all DataFrames in slopes_df
all_slopes_df = pd.concat(slopes_df)
# Create a line plot for all slopes
line_fig = px.line(all_slopes_df, x='delta_b', y='loss', color='Series', hover_data=['delta_b', 'delta_s', 'loss'], markers=True)

# Add each line trace individually
for trace in line_fig.data:
    trace.update(marker=dict(symbol='circle-open', size=10)) 
    fig.add_trace(trace)

# Update marker properties to have darker outlines
# fig.update_traces(marker=dict(line=dict(width=1, color='rgba(40,40,40,1)')))  # Adjust the color as needed

# Update marker properties to remove outlines or match the fill color
fig.update_traces(marker=dict(line=dict(width=0)))  # Removes the outline

# Alternatively, to set the outline to the same color as the fill:
# fig.update_traces(marker=dict(line=dict(width=1, color='SAME_COLOR')))


# Update the layout for dark mode
fig.update_layout(
    title='Scatter Plot in Dark Mode',
    plot_bgcolor='rgb(17,17,17)',  # Dark plot background
    paper_bgcolor='rgb(17,17,17)',  # Dark paper background
    font_color='white',  # White text for better contrast
    xaxis=dict(
        gridcolor='rgb(80,80,80)',  # Darker gridlines
    ),
    yaxis=dict(
        gridcolor='rgb(80,80,80)',  # Darker gridlines
    )
)

# Add plot titles and labels
fig.update_layout(title='effective loss by delta buying',
                  xaxis_title='delta b',
                  yaxis_title='loss',
                  # showlegend=False,
                  height=1600)
                  

# fig.update_yaxes(range=[combined_df['loss'].min(), combined_df['loss'].max()])  # Adjust the range as needed

# # Sort the DataFrame by 'ratio' in descending order
# sorted_df = combined_df.sort_values(by='loss', ascending=False)

# # Adding a line that connects the dots in decreasing order of their y-value
# fig.add_trace(px.line(sorted_df, x='delta_b', y='loss').data[0])

# # Find the five lowest unique y-values (ratios)
# lowest_ratios = combined_df['loss'].unique()
# lowest_ratios.sort()

# # Group by 'ratio' and draw lines for each of the five lowest ratios
# for loss in lowest_ratios[:20]:
#     group = combined_df[combined_df['loss'] == loss]
#     # Sort each group by 'delta_b' to make line connections logical
#     sorted_group = group.sort_values(by='delta_b')
#     fig.add_trace(px.line(sorted_group, x='delta_b', y='loss').data[0])

# # Adjusting the plot size and layout
# fig.update_layout(title='Scatter Plot with Lines for Lowest Y-values',
#                   xaxis_title='Delta_b',
#                   yaxis_title='loss',
#                   height=800)

# Show the plot
fig.show()


In [676]:
m_b / m_s

1.9487171

In [671]:
# Create an interactive scatter plot
fig = px.scatter(combined_df, x='delta_s', y='loss', color='Series', #size='point_size',
                 hover_data=['delta_b', 'delta_s', 'loss'])

# Add lines for specified series
# for series in slopes_df:
#     fig.add_trace(px.line(series, x='delta_s', y='loss', color='Series').data[0])

# Concatenate all DataFrames in slopes_df
all_slopes_df = pd.concat(slopes_df)
# Create a line plot for all slopes
line_fig = px.line(all_slopes_df, x='delta_s', y='loss', color='Series', hover_data=['delta_b', 'delta_s', 'loss'], markers=True)

# Add each line trace individually
for trace in line_fig.data:
    trace.update(marker=dict(symbol='circle-open', size=10)) 
    fig.add_trace(trace)

# Update marker properties to have darker outlines
# fig.update_traces(marker=dict(line=dict(width=1, color='rgba(40,40,40,1)')))  # Adjust the color as needed

# Update marker properties to remove outlines or match the fill color
fig.update_traces(marker=dict(line=dict(width=0)))  # Removes the outline

# Alternatively, to set the outline to the same color as the fill:
# fig.update_traces(marker=dict(line=dict(width=1, color='SAME_COLOR')))


# Update the layout for dark mode
fig.update_layout(
    title='Scatter Plot in Dark Mode',
    plot_bgcolor='rgb(17,17,17)',  # Dark plot background
    paper_bgcolor='rgb(17,17,17)',  # Dark paper background
    font_color='white',  # White text for better contrast
    xaxis=dict(
        gridcolor='rgb(80,80,80)',  # Darker gridlines
    ),
    yaxis=dict(
        gridcolor='rgb(80,80,80)',  # Darker gridlines
    )
)

# Add plot titles and labels
fig.update_layout(title='effective loss by delta selling',
                  xaxis_title='delta s',
                  yaxis_title='loss',
                  # showlegend=False,
                  height=1600)
                  

# fig.update_yaxes(range=[combined_df['loss'].min(), combined_df['loss'].max()])  # Adjust the range as needed

# # Sort the DataFrame by 'ratio' in descending order
# sorted_df = combined_df.sort_values(by='loss', ascending=False)

# # Adding a line that connects the dots in decreasing order of their y-value
# fig.add_trace(px.line(sorted_df, x='delta_b', y='loss').data[0])

# # Find the five lowest unique y-values (ratios)
# lowest_ratios = combined_df['loss'].unique()
# lowest_ratios.sort()

# # Group by 'ratio' and draw lines for each of the five lowest ratios
# for loss in lowest_ratios[:20]:
#     group = combined_df[combined_df['loss'] == loss]
#     # Sort each group by 'delta_b' to make line connections logical
#     sorted_group = group.sort_values(by='delta_b')
#     fig.add_trace(px.line(sorted_group, x='delta_b', y='loss').data[0])

# # Adjusting the plot size and layout
# fig.update_layout(title='Scatter Plot with Lines for Lowest Y-values',
#                   xaxis_title='Delta_b',
#                   yaxis_title='loss',
#                   height=800)

# Show the plot
fig.show()


In [672]:
def prime_factors(n):
    i = 2
    factors = {}
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors[i] = factors.get(i, 0) + 1
    if n > 1:
        factors[n] = factors.get(n, 0) + 1
    return factors

In [673]:
prime_factors(m_b)

{11: 7}

In [674]:
prime_factors(m_s)

{2: 7, 5: 7}

In [675]:
351 - 68

283